In [1]:
import tensorflow as tf

#TODO: Make model more generic. Need to abstractions
class RowlessModel(object):
    # Create input placeholder for the network
    def __init__(self, wordvecdim, num_units, seq1, seq2=None, seq3=None, mode="train"):
        if mode=="train":
            assert seq2 is not None
            assert seq3 is not None

        self.wordvec_dim  = wordvecdim
        self.num_units = num_units
        self.seq_len1 = seq1
        self.seq_len2 = seq2
        self.seq_len3 = seq3
        self.create_placeholders()
        self.create_lstm_outputs()
        self.loss()
        self.sess = tf.Session()
        self.init = tf.global_variables_initializer()
        self.sess.run(self.init)

        # generate the outputs for each input
    def create_lstm_outputs(self):
        with tf.variable_scope('shared_lstm') as scope:
            self.out_input_1 = self.lstm_share(self.num_units, self.input_1, self.seq_len1)
            scope.reuse_variables()  # the variables will be reused.
            self.out_input_2 = self.lstm_share(self.num_units, self.input_2, self.seq_len2, True)
            scope.reuse_variables()
            self.out_input_3 = self.lstm_share(self.num_units, self.input_3, self.seq_len3, True)

    def create_placeholders(self):
        self.input_1 = tf.placeholder(tf.float32, [None, None, self.wordvec_dim], name="s1")
        self.input_2 = tf.placeholder(tf.float32, [None, None, self.wordvec_dim], name = "s2")
        self.input_3 = tf.placeholder(tf.float32, [None, None, self.wordvec_dim], name = "s3")

    # create a shared rnn layer
    def lstm_share(self, num_units, input, seq_len, reuse=False):
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=num_units, state_is_tuple=True, reuse=reuse)
        outputs, _ = tf.nn.dynamic_rnn(cell=lstm_cell,
                                      inputs=input,
                                      sequence_length=seq_len,
                                      dtype=tf.float32
                                      )

        return outputs[-1]

    def loss(self):
        self.loss = tf.log(tf.sigmoid(tf.matmul(self.out_input_1, self.out_input_2) - tf.matmul(self.out_input_1, self.out_input_3)))

    def train(self):
        pass
    

def preprocess_sents():
    

    # # verify whether the variables are reused
    # for v in tf.global_variables():
    #    print(v.name)
    #
    # # concat the three outputs
    # output = tf.concat...
    #
    # # Pass it to the final_lstm layer and out the logits
    # logits = final_layer(output, ...)
    #
    # train_op = ...
    #
    # # train
    # sess.run(train_op, feed_dict{input_1: in1, input_2: in2, input_3:in3, labels: ...}



r = RowlessModel(12,20,[2,3,4],[4,6,7],[1,2,3])
print ("OK")

OK


In [2]:
tf.__version__

'1.4.0'

In [3]:
import codecs
import numpy as np
import pandas as pd
import re

In [75]:
def preprocess_file(f):
    """Returns np.array [9 x n_sents]. Columns : [e1,e1_str,e1_start_idx,e1_end_idx,e2,e2_str,e2_start_idx,e2_end_idx,sent]"""
    with open(f,'rb') as f:
        test_lines = [(str(codecs.unicode_escape_decode(str(i)[2:-3])[0])[1:].split('\t')) for i in f.readlines()]
    test_lines = np.array([i for i in test_lines if len(i)==13])
    ip = test_lines.T
    ip[12] = np.array([re.sub('[^\w\s]','',i.lower()) for i in ip[12]])
    return ip[[0,2,3,4,5,7,8,9,12]]

In [46]:
df = pd.DataFrame({'e1':ip[0],'e2':ip[4],'e1_str':ip[2],'e2_str':ip[7],'sent':ip[12],\
              'e1_idx_s':ip[3],'e1_idx_e':ip[4],'e2_idx_s':ip[8],'e2_idx_e':ip[9]})

In [58]:
f = 'D:\\Comp\\689_project\\data\\kb_00.txt'
prproc = preprocess_file(f)